# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 60 new papers today
          24 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/23 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.07723


extracting tarball to tmp_2301.07723...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.07724


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2301.07723/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'citation_fix' from 'tmp_2301.07723/citation_fix.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

extracting tarball to tmp_2301.08162...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.08192


extracting tarball to tmp_2301.08192...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2301.08192/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.07990-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.07990) | **Timing the formation of the Galactic thin disc with asteroseismic  stellar ages**  |
|| Wu Yaqian, et al. -- incl., <mark>Zhao Gang</mark>, <mark>Li Yaguang</mark> |
|*Appeared on*| *2023-01-20*|
|*Comments*| *17 pages, 13 figures, accepted by MNRAS*|
|**Abstract**| The formation of the extended thin disc is the most spectacular event of our Galaxy in the past $\sim8$\,Gyr. To unveil this process, obtaining precise and accurate stellar ages for a large sample of stars is essential although challenging. In this work, we present the asteroseismic age determination of 5306 red giant branch stars using \kepler{} and LAMOST data, with a thorough examination of how the age determination is affected by the choice of different temperature scales and stellar models. Thanks to the high precision of the asteroseismic and spectroscopic parameters of our sample stars, we are able to achieve age determination with an average accuracy of 12 per cent. However, the age determination is sensitively dependent on the adopted temperature scale, as 50\,K difference in effective temperature may cause larger than 10 per cent systematic uncertainty in the age estimates. Using the ages derived with the most plausible set of the temperature scale, we study the age distribution of the chemical thin disc stars, and present an estimate of the formation epoch of the first Galactic thin disc stars. We find that the first (oldest) thin disc stars have an age of $9.5^{+0.5(\rm rand.)+0.5(\rm sys.)}_{-0.4(\rm rand.)-0.3(\rm sys.)}$\,Gyr, where the systematic uncertainties reflect ages estimated using different stellar evolutionary models. At this epoch, the Galactic thick disc was still forming stars, indicating there is a time window when both the thin and thick discs of our Galaxy were forming stars together. Moreover, we find that the first thin disc stars exhibit a broad distribution of Galactocentric radii, suggesting that the inner and outer thin discs began to form simultaneously. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.07724-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.07724) | **The SOUX AGN Sample: SDSS-XMM-Newton Optical, Ultraviolet and X-ray  selected active galactic nuclei spanning a wide range of parameter space --  Sample definition**  |
|| Daniel Kynoch, et al. -- incl., <mark>Hermine Landt</mark> |
|*Appeared on*| *2023-01-20*|
|*Comments*| *23 pages, 24 figures and 10 tables. Accepted for publication in MNRAS*|
|**Abstract**| We assemble a sample of 696 type 1 AGN up to a redshift of $z=2.5$, all of which have an SDSS spectrum containing at least one broad emission line (H $\alpha$, H $\beta$ or Mg II) and an XMM-Newton X-ray spectrum containing at least 250 counts in addition to simultaneous optical/ultraviolet photometry from the XMM Optical Monitor. Our sample includes quasars and narrow-line Seyfert 1s: thus our AGN span a wide range in luminosity, black hole mass and accretion rate. We determine single-epoch black hole mass relations for the three emission lines and find that they provide broadly consistent mass estimates whether the continuum or emission line luminosity is used as the proxy for the broad emission line region radius. We explore variations of the UV/X-ray energy index $\alpha_\mathrm{ox}$ with the UV continuum luminosity and with black hole mass and accretion rate, and make comparisons to the physical quasar spectral energy distribution (SED) model QSOSED. The majority of the AGN in our sample lie in a region of parameter space with $0.02<L/L_\mathrm{Edd}<2$ as defined by this model, with narrow-line type 1 AGN offset to lower masses and higher accretion rates than typical broad-line quasars. We find differences in the dependence of $\alpha_\mathrm{ox}$ on UV luminosity between both narrow/broad-line and radio-loud/quiet subsets of AGN: $\alpha_\mathrm{ox}$ has a slightly weaker dependence on UV luminosity for broad-line AGN and radio-loud AGN have systematically harder $\alpha_\mathrm{ox}$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.07993-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.07993) | **Ammonia mapping observations of the Galactic infrared bubble N49: Three  NH$_3$ clumps along the molecular filament**  |
|| Mikito Kohno, et al. -- incl., <mark>Ross A. Burns</mark> |
|*Appeared on*| *2023-01-20*|
|*Comments*| *29 pages, 13 figures, 3 tables, accepted for Publications of the Astronomical Society of Japan (PASJ)*|
|**Abstract**| We have carried out the NH$_3$ $(J,K)=(1,1),(2,2),$ and $(3,3)$ mapping observations toward the Galactic infrared bubble N49 (G28.83-0.25) using the Nobeyama 45 m telescope. Three NH$_3$ clumps (A, B, and C) were discovered along the molecular filament with the radial velocities of $\sim$ 96, 87, and 89 km s$^{-1}$, respectively. The kinetic temperature derived from the NH$_3$ (2,2)/NH$_3$ (1,1) shows $T_{\rm kin} = 27.0 \pm 0.6$ K enhanced at Clump B in the eastern edge of the bubble, where position coincides with massive young stellar objects (MYSOs) associated with the 6.7 GHz class II methanol maser source. This result shows the dense clump is locally heated by stellar feedback from the embedded MYSOs. The NH$_3$ Clump B also exists at the 88 km s$^{-1}$ and 95 km s$^{-1}$ molecular filament intersection. We therefore suggest that the NH$_3$ dense gas formation in Clump B can be explained by a filament-filament interaction scenario. On the other hand, NH$_3$ Clump A and C at the northern and southern side of the molecular filament might be the sites of spontaneous star formation because these clumps are located $\sim$5$-$10 pc away from the edge of the bubble. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.08048-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.08048) | **CRIRES$^{+}$ on sky at the ESO Very Large Telescope**  |
|| R. J. Dorn, et al. -- incl., <mark>U. Heiter</mark>, <mark>B. Klein</mark>, <mark>J-L. Lizon</mark> |
|*Appeared on*| *2023-01-20*|
|*Comments*| *Accepted for publication in Astronomy and Astrophysics*|
|**Abstract**| The CRyogenic InfraRed Echelle Spectrograph (CRIRES) Upgrade project CRIRES$^{+}$ extended the capabilities of CRIRES. It transformed this VLT instrument into a cross-dispersed spectrograph to increase the wavelength range that is covered simultaneously by up to a factor of ten. In addition, a new detector focal plane array of three Hawaii 2RG detectors with a 5.3 $\mu$m cutoff wavelength replaced the existing detectors. Amongst many other improvements, a new spectropolarimetric unit was added and the calibration system has been enhanced. The instrument was installed at the VLT on Unit Telescope 3 at the beginning of 2020 and successfully commissioned and verified for science operations during 2021, partly remotely from Europe due to the COVID-19 pandemic. The instrument was subsequently offered to the community from October 2021 onwards. This article describes the performance and capabilities of the upgraded instrument and presents on sky results. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.08052-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.08052) | **The Planck clusters in the LOFAR sky: IV: LoTSS-DR2: statistics of radio  halos and re-acceleration models**  |
|| R. Cassano, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2023-01-20*|
|*Comments*| *12 pages, 10 figures, accepted for publication in A&A*|
|**Abstract**| Diffuse cluster-scale synchrotron radio emission is discovered in an increasing number of galaxy clusters in the form of radio halos (RHs), probing the presence of relativistic electrons and magnetic fields in the intra-cluster medium. The favoured scenario to explain their origin is that they trace turbulent regions generated during cluster mergers where particles are re-accelerated. In this framework, RHs are expected to probe cluster dynamics and are predicted to be more frequent in massive systems. Statistical studies are important to study the connection of RHs with cluster dynamics and to constrain theoretical models. Furthermore, low-frequency surveys can shed light on the existence of RHs with very steep radio-spectra, a key prediction of turbulent models. We study the properties of RHs from clusters of the second catalog of Planck Sunyaev Zel'dovich detected sources that lie within the 5634 deg^2 covered by the second Data Release (DR2) of the LOFAR Two-meter Sky Survey. We find that the number of observed RHs, their radio flux density and redshift distributions are in line with what is expected in the framework of the re-acceleration scenario. In addition, the fraction of clusters with RHs increases with the cluster mass, confirming the leading role of the gravitational process of cluster formation in the generation of RHs. These models predict a large fraction of RHs with very steep spectrum in the DR2 Planck sample, this will be tested in future studies, yet a comparison of the occurrence of halos in GMRT and LOFAR samples indeed shows a larger occurrence of RHs at lower frequencies suggesting the presence of a number of very steep spectrum RH that is preferentially detected by LOFAR. Using morphological information we confirm that RHs are preferentially located in merging systems and that the fraction of newly LOFAR discovered RHs is larger in less disturbed systems. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.08162-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.08162) | **Characterization of a set of small planets with TESS and CHEOPS and an  analysis of photometric performance**  |
|| Dominic Oddo, et al. -- incl., <mark>Mallory Harris</mark>, <mark>Richard P. Schwarz</mark>, <mark>Guillermo Torres</mark>, <mark>Joseph D. Twicken</mark>, <mark>Carl Ziegler</mark> |
|*Appeared on*| *2023-01-20*|
|*Comments*| *Accepted for publication in AJ*|
|**Abstract**| The radius valley carries implications for how the atmospheres of small planets form and evolve, but this feature is only visible with highly-precise characterizations of many small planets. We present the characterization of 9 planets and one planet candidate with both NASA TESS and ESA CHEOPS observations, which adds to the overall population of planets bordering the radius valley. While five of our planets - TOI 118 b, TOI 262 b, TOI 455 b, TOI 560 b, and TOI 562 b - have already been published, we vet and validate transit signals as planetary using followup observations for four new TESS planets, including TOI 198 b, TOI 244 b, TOI 444 b, and TOI 470 b. While a three times increase in primary mirror size should mean that one CHEOPS transit yields an equivalent model uncertainty in transit depth as about nine TESS transits under ideal conditions, we find that our CHEOPS transits typically yield uncertainties equivalent to between 2 and 12 TESS transits, averaging 5.9 equivalent transits. Therefore, we find that while our fits to CHEOPS transits provide overall lower uncertainties on transit depth and better precision relative to fits to TESS transits, our uncertainties for these fits do not always match expected predictions given photon-limited noise. We find no correlations between number of equivalent transits and any physical parameters, indicating that this behavior is not strictly systematic, but rather might be due to other factors such as in-transit gaps during CHEOPS visits or non-homogeneous detrending of CHEOPS light curves. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.07760-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.07760) | **Astronomical Detection of the Interstellar Anion C10H- towards TMC-1  from the GOTHAM Large Program on the GBT**  |
|| Anthony Remijan, et al. -- incl., <mark>Eric Herbst</mark>, <mark>Kin Long Kelvin Lee</mark> |
|*Appeared on*| *2023-01-20*|
|*Comments*| *38 Pages, 24 Figures, 12 Tables, 8 Appendices*|
|**Abstract**| Using data from the GOTHAM (GBT Observations of TMC-1: Hunting for Aromatic Molecules) survey, we report the first astronomical detection of the C10H- anion. The astronomical observations also provided the necessary data to refine the spectroscopic parameters of C10H-. From the velocity stacked data and the matched filter response, C10H- is detected at >9{\sigma} confidence level at a column density of 4.04e11 cm-2. A dedicated search for the C10H radical was also conducted towards TMC-1. In this case, the stacked molecular emission of C10H was detected at a ~3.2{\sigma} confidence interval at a column density of 2.02e11 cm-2. However, since the determined confidence level is currently <5{\sigma}, we consider the identification of C10H as tentative. The full GOTHAM dataset was also used to better characterize the physical parameters including column density, excitation temperature, linewidth, and source size for the C4H, C6H and C8H radicals and their respective anions, and the measured column densities were compared to the predictions from a gas/grain chemical formation model and from a machine learning analysis. Given the measured values, the C10H-/C10H column density ratio is ~2.0 - the highest value measured between an anion and neutral species to date. Such a high ratio is at odds with current theories for interstellar anion chemistry. For the radical species, both models can reproduce the measured abundances found from the survey; however, the machine learning analysis matches the detected anion abundances much better than the gas/grain chemical model, suggesting that the current understanding of the formation chemistry of molecular anions is still highly uncertain. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.08071-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.08071) | **The GRAVITY+ Project: Towards All-sky, Faint-Science, High-Contrast  Near-Infrared Interferometry at the VLTI**  |
|| GRAVITY+ Collaboration, et al. -- incl., <mark>Helen Bedigan</mark>, <mark>Henri Bonnet</mark>, <mark>Ric Davies</mark>, <mark>Frank Eisenhauer</mark>, <mark>Helmut Feuchtgruber</mark>, <mark>Natascha M. Förster Schreiber</mark> |
|*Appeared on*| *2023-01-20*|
|*Comments*| *Published in the ESO Messenger*|
|**Abstract**| The GRAVITY instrument has been revolutionary for near-infrared interferometry by pushing sensitivity and precision to previously unknown limits. With the upgrade of GRAVITY and the Very Large Telescope Interferometer (VLTI) in GRAVITY+, these limits will be pushed even further, with vastly improved sky coverage, as well as faint-science and high-contrast capabilities. This upgrade includes the implementation of wide-field off-axis fringe-tracking, new adaptive optics systems on all Unit Telescopes, and laser guide stars in an upgraded facility. GRAVITY+ will open up the sky to the measurement of black hole masses across cosmic time in hundreds of active galactic nuclei, use the faint stars in the Galactic centre to probe General Relativity, and enable the characterisation of dozens of young exoplanets to study their formation, bearing the promise of another scientific revolution to come at the VLTI. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2301.07990.md
    + _build/html/tmp_2301.07990/./timing_cleanfig9.png
    + _build/html/tmp_2301.07990/./timing_cleanfig3.png
    + _build/html/tmp_2301.07990/./timing_cleanfig4.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\kepler}[0]{\emph{Kepler}}$
$\newcommand{\gaia}[0]{\emph{Gaia}}$
$\newcommand{\mmode}[1]{\ifmmode{#1}\else{#1}\fi}$
$\newcommand{\Dnu}[0]{\mmode{\Delta\nu}}$
$\newcommand{\numax}[0]{\mmode{\nu_\text{max}}}$
$\newcommand{\Teff}[0]{\mmode{T_\text{eff}}~}$
$\newcommand{\liyg}[1]{{\color{orange} #1}}$
$\newcommand$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\kepler$}[0]{\emph{Kepler}}$
$\newcommand{$\gaia$}[0]{\emph{Gaia}}$
$\newcommand{$\mmode$}[1]{\ifmmode{#1}\else{#1}\fi}$
$\newcommand{$\Dnu$}[0]{$\mmode${\Delta\nu}}$
$\newcommand{$\numax$}[0]{$\mmode${\nu_\text{max}}}$
$\newcommand{$\Teff$}[0]{$\mmode${T_\text{eff}}~}$
$\newcommand{$\liyg$}[1]{{\color{orange} #1}}$
$\newcommand$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Timing the formation of the Galactic thin disc with asteroseismic stellar ages

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.07990-b31b1b.svg)](https://arxiv.org/abs/2301.07990)<mark>Appeared on: 2023-01-20</mark> - _17 pages, 13 figures, accepted by MNRAS_

</div>
<div id="authors">

Wu Yaqian, et al. -- incl., <mark><mark>Zhao Gang</mark></mark>, <mark><mark>Li Yaguang</mark></mark>

</div>
<div id="abstract">

**Abstract:** The formation of the extended thin disc is the most spectacular event of our Galaxy in the past$\sim8$Gyr. To unveil this process, obtaining precise and accurate stellar ages for a large sample of stars is essential although challenging.In this work, we present the asteroseismic age determination of 5306 red giant branch stars using$\kepler$and LAMOST data, with a thorough examination of how the age determination is affected by the choice of different temperature scales and stellar models. Thanks to the high precision of the asteroseismic and spectroscopic parameters of our sample stars, we are able to achieve age determination with an average accuracy of 12 per cent. However, the age determination is sensitively dependent on the adopted temperature scale, as 50 K difference in effective temperature may cause larger than 10 per cent systematic uncertainty in the age estimates. Using the ages derived with the most plausible set of the temperature scale, we study the age distribution of the chemical thin disc stars, and present an estimate of the formation epoch of the first Galactic thin disc stars. We find that the first (oldest) thin disc stars have an age of$9.5^{+0.5(\rm rand.)+0.5(\rm sys.)}_{-0.4(\rm rand.)-0.3(\rm sys.)}$Gyr, where the systematic uncertainties reflect ages estimated using different stellar evolutionary models. At this epoch, the Galactic thick disc was still forming stars, indicating there is a time window when both the thin and thick discs of our Galaxy were forming stars together. Moreover, we find that the first thin disc stars exhibit a broad distribution of Galactocentric radii, suggesting that the inner and outer thin discs began to form simultaneously.

</div>

<div id="div_fig1">

<img src="tmp_2301.07990/./timing_cleanfig9.png" alt="Fig9" width="100%"/>

**Figure 9. -** Age distribution histogram for the thin (black) and thick disc (grey) stars. The red line and squares represent the mock data set that best fits the the observed age histogram for determining the upper age bound of the thin disc stars (see main text). The fit suggests an upper bound of 9.5 Gyr for the intrinsic ages of the thin disc stars. (*fig6*)

</div>
<div id="div_fig2">

<img src="tmp_2301.07990/./timing_cleanfig3.png" alt="Fig3" width="100%"/>

**Figure 3. -** {\em left}: Differential temperature between the LAMOST DD-Payne estimates and the photometric estimates using the \citet{Gonz2009} temperature scale as a function of [Fe/H] with colour-coded stellar number density.
    The solid line in black shows a linear fit to the difference for the disc stars, and it yields a negative trend of $\Delta$Teff= -54.9690 - 86.8584$\times$[Fe/H]. {\em Right}: Same as the left panel, but for temperature difference between the LAMOST DD-Payne estimates and the photometric estimates using the \citet{Huang2015} temperature scale. A linear fit (black) is found to well reproduce the systematic trend for the disc stars: $\Delta$Teff= 29.8236 - 27.8769$\times$[Fe/H]. (*fig:fig2*)

</div>
<div id="div_fig3">

<img src="tmp_2301.07990/./timing_cleanfig4.png" alt="Fig4" width="100%"/>

**Figure 4. -** {\em Left}: Distributions of age estimates, as well as their error estimates, for the RGB sample stars. {\em Right}: Spatial distribution of the RGB sample stars with colour-coded ages in the $R$ and $Z$ plane. (*fig4*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

176  publications in the last 7 days.
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/html/2301.04533.md
	 _build/html/2301.04442.md
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/html/2301.01937.md
	 _build/html/2301.01895.md
	 _build/html/2301.01684.md
	 _build/html/2301.01306.md
	 _build/html/2301.00881.md
	 _build/html/2301.00854.md
	 _build/html/2301.00578.md
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers